In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import math
import torch
import pickle
import numpy as np
import pandas as pd
import torch.nn as nn
from datetime import datetime, timedelta
from sklearn.metrics import mean_squared_error

from torch_geometric.nn import GCNConv
import torch.nn.functional as F

In [ ]:
directory = '/content/drive/MyDrive/Thesis'
data_dir = directory + "/Data"
models_dir = directory + "/models"

In [ ]:
timestamps = np.load(f'{data_dir}/demand_graph_timestamps.pkl.npz')
stamps = [timestamps[f'arr_{i}'] for i in range(8752, len(timestamps))]

In [ ]:
station_clusters = [
    74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,  86,
    87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99,
    100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112,
    113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
    126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138,
    139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151,
    152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164,
    165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177,
    178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190,
    191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203,
    205, 206, 208, 209, 210, 211, 212, 213, 214, 216, 217, 218, 219,
    220, 222, 223, 224, 225, 226, 227, 228, 230, 231, 232, 233, 234,
    235, 237, 238, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249,
    250, 251, 252, 253, 255, 257, 258, 261, 264, 265, 266, 274, 275,
    278
]

num_stations = len(station_clusters)

In [ ]:
loaded = np.load(f'{data_dir}/demand_graphs.pkl.npz')
demand_graphs = [loaded[f'arr_{i}'] for i in range(8752, len(loaded))]
adj_matrices = torch.tensor(demand_graphs)
del demand_graphs



<ipython-input-7-0ad896c72982>:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  adj_matrices = torch.tensor(demand_graphs)


In [ ]:
!pip install torch_geometric

In [ ]:
from torch_geometric.data import Data
from torch_geometric.utils import dense_to_sparse

# Convert adjacency matrices to PyTorch Geometric format for each hour
data_list = []

for i in range(adj_matrices.shape[0]):  # Iterate over each hour
    adj_matrix = adj_matrices[i]

    # Convert dense adjacency matrix to edge_index (sparse representation)
    edge_index, edge_attr = dense_to_sparse(adj_matrix)
    # Use an identity matrix for node features (can be replaced with other features)
    x = torch.eye(num_stations)
    # Create a Data object
    data = Data(x=adj_matrix.float(), edge_index=edge_index, edge_attr=edge_attr)
    data_list.append(data)

print(data_list[0])

for data in data_list:
    data.edge_attr = data.edge_attr.float()

Data(x=[183, 183], edge_index=[2, 47], edge_attr=[47])


In [ ]:
del adj_matrices

In [ ]:
n_rows = len(target_array)

target = target_array.reshape(-1, num_stations)

In [ ]:
stamps[8765][0]

array([2.024e+03, 1.000e+00, 1.000e+00, 0.000e+00, 0.000e+00])

In [ ]:
train_target = target[3:8765]
test_target = target[8765:]

In [ ]:
len(test_target)

2183

In [ ]:
len(data_list[2:8764]) #train
len(data_list[8764:-1]) #test

2183

In [ ]:
class GNNLayer(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim):
      super(GNNLayer, self).__init__()
      self.conv1 = GCNConv(input_dim, hidden_dim)
      self.conv2 = GCNConv(hidden_dim, hidden_dim)
      self.relu = nn.ReLU()

    def forward(self, data):
      x, edge_index = data.x, data.edge_index
      x = self.relu(self.conv1(x, edge_index))
      x = self.relu(self.conv2(x, edge_index))
      return x

class GNNLSTM(torch.nn.Module):
    def __init__(self, in_channels, final_output_dim, output_dim=100, lstm_hidden_dim=128, num_layers=3, dropout_prob=0.2):
        super(GNNLSTM, self).__init__()
        self.gnn = GNNLayer(in_channels, output_dim)
        self.lstm = nn.LSTM(output_dim, lstm_hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(lstm_hidden_dim, final_output_dim)

    def forward(self, data):

      node_embeddings = []
      for data in data_list:
        node_embeddings.append(self.gnn(data))


      # Stack node embeddings across time (batch_size, sequence_length, num_nodes, hidden_channels)
      node_embeddings = torch.stack(node_embeddings, dim=1)  # Shape: [batch_size, seq_len, num_nodes, hidden_channels]

      # Step 2: Apply LSTM on node embeddings over time
      #print(node_embeddings.shape)
      seq_len, num_nodes, hidden_dim = node_embeddings.shape
      node_embeddings = node_embeddings.view(num_nodes, seq_len, hidden_dim)  # Flatten nodes and time

      lstm_out, _ = self.lstm(node_embeddings)  # Pass through LSTM, output shape: [batch_size * num_nodes, seq_len, lstm_hidden_size]

      # Step 3: Predict demand using the last LSTM output
      lstm_out = lstm_out[:, -1, :]  # Take the output from the last time step
      #lstm_out = lstm_out.view(batch_size, num_nodes, -1)  # Reshape back to (batch_size, num_nodes, lstm_hidden_size)

      # Final demand prediction from the node embeddings
      predicted_demand = self.fc(lstm_out)  # Shape: [batch_size, num_nodes, out_channels]
      #print(predicted_demand)
      return predicted_demand

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [ ]:
train_labels = torch.tensor(train_target, dtype=torch.float32)
test_labels = torch.tensor(test_target, dtype=torch.float32)

In [ ]:
embedding_dim = 50
hidden_dim = 128
num_layers = 3

# LSTM model
model = GNNLSTM(
    in_channels=num_stations, final_output_dim=num_stations,
    output_dim=embedding_dim, lstm_hidden_dim=hidden_dim, num_layers=num_layers
  ).to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)

epochs = 10
# Training loop
model.train()
for epoch in range(epochs):  # Number of epochs
    for inputs, targets in zip(data_list, train_labels):
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, targets)  # Loss calculation
        loss.backward()  # Backpropagation

        # Clip gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()  # Update weights

    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

In [ ]:
inputs.x

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [ ]:
dataset = SequenceDataset(test_embedding, test_labels)
test_dataloader = DataLoader(dataset, batch_size=48, shuffle=True)

NameError: name 'test_embedding' is not defined

In [ ]:
preds = []
model.eval()
with torch.no_grad():
    for inputs, targets in test_dataloader:
        inputs = inputs.to(device)
        predictions = model(inputs)  # Shape: [batch_size, output_dim]
        preds.extend(predictions.reshape(-1))

In [ ]:
all_tests = test_labels.reshape(-1)

In [ ]:
predictions

tensor([[ 2.2160e-01,  7.2856e-01,  9.3312e-01,  ..., -6.1146e-08,
          2.5188e-12,  1.5824e-06],
        [ 3.0785e-01,  1.0109e+00,  1.2486e+00,  ..., -7.4423e-08,
          1.6979e-12,  8.1800e-07],
        [ 3.6023e-01,  1.2128e+00,  1.4402e+00,  ..., -7.6610e-08,
          1.4282e-12,  6.2063e-07],
        ...,
        [ 1.9510e-01,  6.1997e-01,  8.1353e-01,  ..., -4.6711e-08,
          2.7647e-12,  1.4031e-06],
        [ 3.8940e-01,  1.3252e+00,  1.5435e+00,  ..., -6.9205e-08,
          1.4055e-12,  4.8114e-07],
        [ 4.0848e-02,  1.0548e-01,  1.9283e-01,  ..., -1.9408e-07,
         -1.6586e-12,  8.4182e-07]])

In [ ]:
rmse_score = np.sqrt(mean_squared_error(preds, all_tests))

In [ ]:
rmse_score

2.5110908